In [2]:
#DEBUG
%load_ext autoreload
%autoreload 2

from bokeh.io import output_notebook, show
output_notebook(hide_banner=True);

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import plot
from Signal import Signal
from ipywidgets import interact



from utils import tag_cell, tag_code_input, tag_markdown_cell, code_horizontal, code_horizontal_reverse, set_css_style
display(set_css_style('./styles/custom.css'))
code_horizontal([0,1]);
tag_cell([5, 6, 8, 9, 11, 12], 'half_width')
code_horizontal_reverse([10]);
# tag_cell([0, 1, 9, 13, 70, 73, 91, 94], 'full_width');
# tag_cell([2, 3, 4, 5, 7, 8, 11, 12, 14, 15, 17, 18, 22, 23, 
#           27, 28, 32, 33, 35, 36, 37, 38, 44, 45, 47, 48, 52, 
#           53, 55, 56, 58, 59, 61, 62, 64, 65, 67, 68, 71, 72,
#          75, 76, 78, 79, 81, 82, 84, 85, 89, 90, 95, 96, 97, 98], 'half_width');

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
// Used to ensure figures take as much space as they need and not introduce a scroll bar

<IPython.core.display.Javascript object>

# __Background and Introduction to Fourier Analysis in Audio__

## What is Fourier Analysis?

<br>

Fourier analysis is the study of how we can represent general functions such as the line on the right as a sum of simpler trignometric functions. This lets us look at functions from a different point of view, giving us a new perspective on a function. We'll see later how we can use it to decompose a sound and see the frequency components that it composes of. 

<br>

It is used in numerous fields and extends far beyond audio analysis which is where we will explore it from. 

<br>

Before we get there though we have to understand some basic properties about trignometric functions and signals. How do we represent them and how are they stored digitally?

![FourierSeries](images/fourierseries.svg.png)

## __Overview__

<br>

#### Simple Signals and Background
* __Trigonometric functions__
    * Sin and Cos
    * Moving to the _Time_ domain
    * Adding functions
* __Signals__
    * Sound Signals
    * _Discrete_ vs _Continuous_
    * Sampling & Limits
    * Aliasing 
    * Noise

<br>

#### Detecting Signals
* __Frequency Estimation, the problem__
    * Creating a random signal
    * Frequency estimation
* __Dot product__
    * Vectors
    * Dot product operation
    * Normalization
* __Building a Detector, Part 1__
    * Using the dot product
    * Composite Signals
    * Noisy signals
* __Dealing with Phase__
    * Phase
    * Identifying the problem
    * Forming a solution
    * Testing our solution
* __Recap__



<br>
<br>

#### A Complex introduction
* __Orthogonality__
* __Complex Numbers__
    * Basic introduction
    * _Rectangular_ vs _Polar Form_
    * Relation to _cos_ and _sin_
    * Phase and angle
    * Complex exponentials
    * Complex conjugates
    * Summary
* __Building a Detector, Part 2__
    * A complex signal
    * Simplifying our detectors work  

<br>

#### Fourier Analysis
* __Fourier Series__
    * What it can do
    * Understanding the coefficients
    * The complex Fourier Series
 
#### TODO:

## __Simple Signals and Background - Trigonometric functions__

## Sin and Cos

A $sin$ or $cos$ wave can be considered one of the building blocks of the analysis we will be doing. You can see an example of a $sin$ wave on the right. It repeats itself every so often and also goes just as high above the line as it does below.

We can call the length from which it start to where it begind to repeat itself its __period__ which we can call $T$, for the $sin$ wave on the right, its period is $2\pi$.

The height above and below the central axis is its __amplitude__ which we can call $A$.

The easiest way to represent all of this is to form an equation: 

$$
x(t) = Asin(t) \quad or \quad x(t) = Acos(t)
$$

The main noticable difference for now is where they start, the red line on the left is a $cos$ wave and you'll notice it start at $A$ where as the $sin$ start at $0$.


This isn't very exciting by itself but we can make them repeat faster, decreasing its _period_ if we want by multipling our $t$ by some number $f$, its __frequency__:

$$
x(t) = Asin(f \cdot t)
$$


For example if we wanted to create a a $sin$ wave that repeats itself $3$ times within the range $[0, 2\pi]$ and has an amplitude of $5$ we could represent that as $x(t) = 5 \cdot sin(3 \cdot t)$. Lets see how we might code that using the Python library [Numpy](http://www.numpy.org/).

<br>

Try changing the values of both `f` and `A` in the code block below if you like to get a better picture.

<img src="images/simple_sin_wave.png" alt="drawing" width="800" height="300" >
<img src="images/sin_and_cos.png" alt="drawing" , width="800" height="300" >

In [ ]:
import numpy as np # A numberical library used extensively in Python
# You might notice some odd behaviour at high f, we'll see this when looking at sampling
f = 3 
A = 2

t = np.linspace(0, 4*np.pi, 48) # Create 256 values evenly spaced between [0, 4pi]
y = A * np.sin(f*t) # Calculate our sin wave

sig = Signal(t, y) # Wrap it up in a Signal Object

sig.show(size=(800, 300), title='A simple Sin wave', animated=True, frames=128, views=['stems']) ;
# Show the signal with option to animate (True or False)

## The relationship between frequency and period

<br>

Our _period_ $T$ and our _frequency_ $f$ are __inversly related__ meaning as we increase our frequency, we should expect the period of our functions to get smaller.

<br>

#### Frequency and Period
$$
\begin{align*}
T & = \frac{1}{f} 
\\
f & = \frac{1}{T}
\end{align*}
$$

## Moving to the Time Domain

<br>

So far we've been talking about waves that have a period in units of $2\pi$, for example with $f = 3$, our wave would repeat $3$ times in the space of $[0, 2\pi]$. When it comes to audio and real life signals, we often talk about frequency in terms of $Hz$ (Hertz) which measure the cycles per second. With $f = 3\textbf{Hz}$ we would instead repeat $3$ times in $[0, 1]$.

<br>

#### Converting between the two

We are essentially mapping the range $[0, 1]$ to $[0, 2\pi]$ as this is the period our trigonometric functions work in. This can be done simply by multiplying our time by $2\pi$, feel free to verify this for yourself.

_i.e. we would like 0 to map to 0, 0.5 to map to $1\pi$ and 1 to map to $2\pi$_

<br>

This gives us a new equation for talking about functions that relate to time, for example when measuring Audio, we generally prefer to measure in seconds and measure frequency in $Hz$ so we might describe a a 10Hz signal as $sin(10 \cdot 2\pi \cdot t)$, completing 10 cycles every second. 


In [ ]:
from plot import scale_time
scale_time(size=(1680, 300), animated=False)  # The purple marker indicates the 1 second marker and where it ends up

## Adding Functions

<br>

We can add functions to create a new function. We'll be doing this to create more intersting periodic functions but lets see what happens when we add the following two functions:

<br>

$$
\begin{align}
x(t) & = f(t) + g(t)
\\
\\
f(t) & = 2t + 4
\\
g(t) & = -t^2 + 2
\end{align}
$$


In [2]:
t = np.linspace(-3, 3, 128)
ft = 2*t + 4
gt = -(t*t) + 2
xt = ft + gt

plot.add_functions(t, ft, gt, xt, animated=True);

### Adding Periodic Function
Now lets try adding two periodic functions in much the same way and and see what we get!

$
x(t) = sin(3 \cdot 2\pi \cdot t) + sin(4 \cdot 2\pi \cdot t)
$

In [4]:
t = np.linspace(0, 4, 256)
f1 = np.sin(3 * 2*np.pi * t) #2*t + 4 - demonstration purposes
f2 = np.sin(4 * 2*np.pi * t)
xt = f1 + f2

plot.add_functions(t, f1, f2, xt, animated=True)


What you might notice is that where the two functions line up, we get a larger spike, either in the positive or negative direction, this is where there respective peaks are lined up. In audio this is called __constructive interference__. Where they don't line up, we get this darker green section where they cancel out and produce a very small amplitude, __destructive interference__. If two periodic functions are of the same frequency, there peaks will line up with eachother necessarily, otherwise we'll always encounter this darker shade of green where destructive interference is happening. _There's also a concept called phase which we'll get too later!_

Another cool thing to notice is that in our added function, the result is also periodic. An intuitive reasoning for this is that, periodic functions will repeat forever, and at somepoint, two periodic functions will begin their periods at exactly the same point! In this case, we've made both our functions periodic around 1 second so we should expect our result to also be periodic every 1 second.

<br>

For example, if we were to have one signal have a period of 4 seconds and another signal have a period of 3 seconds, we know their _lowest common multiple_ is 12. We can expect the result of adding them together to _at least_ be periodic every 12 seconds.

## __Simple Signals and Background - Signals__

## Sound Signals

<br>

#### A brief look at sound

A real life sound is a signal, air molecules get pushed around and the pressure at any one point changes over time. 

![SoundGif](http://i.imgur.com/7ihsGHx.gif)

#### Storing Sound - Digital Signals

<br>

The vibration of air molecules, this push of air molecules is what our ear picks up on and eventually gets converted to something we know as sound. Storing this phenomina digitally requires us to capture details about the air pressure over time as the sound progresses. 

<br>

We might decide that our highest pressure is a $1$ and our lowest possible pressure, none is $0$ and map each second of sound into 1 bit values. So at each moment in time we store the sound as a **0 or a 1**. Lets create a signal the slowly ramps up and see how this fairs.

<br>

![DigitalSignal1bit](images/Digital_signal_1bit.png)

<br>

Well it seems to have missed out on a lot of detail. Lets try a $4$ bit number that might look like **0100**, 4 **bits** that we can set to either a 0 or 1. This would give us $4^2 = 16$ possible values.

<br>

![DigitalSignal4bit](images/Digital_signal_4bit.png)

<br>

Definitely a lot better but still quite janky. The more bits you use the more memory you need to store one sample of sound at some moment in time.

<br>

#### How many bits should we use?

<br> 

WAV files, a form of uncompressed audio are generally stored as 24bits per sample which is more than enough for most applications and good enough for listening.  

<br>

Music is generally contain sounds which are produced by something that oscillates or vibrates. Because of this oscillating nature, we normally record amplitudes of pressure around $0$, where the lowest amplitude is some $-A$ and the highest $+A$. This helps us make use of analysis techniques used for periodic signals in maths accessible to music and gives us the wide range of digital tools we have have avaialable today to process, store and create music. 

<br>

In [5]:
from plot import digitize_sound
max_time = 1
t = np.linspace(0, max_time, 8192)

f = 261 #This is around a C4 by MIDI standard
sound = 30* np.sin(f * 2*np.pi * t)
sound_signal = Signal(t, sound)

digitize_sound(sound_signal, max_bits=12)

Button(description='Play Digital Audio', style=ButtonStyle())

interactive(children=(IntSlider(value=3, description='bits', max=12, min=1), Output()), _dom_classes=('widget-…

## Discrete vs. Continuous Signals
Up until now we've been talking as if we have been using continuous signals. You might have also wondered why we chose to to make `t = np.linspace(0, 1, 64)` to get 64 samples then. As we said when talking about sound, we have to sample it so that we can use it digitally. This also applies to many other continuous signals. 

Consider the problem of recording the temperature outside. It's always fluctuating at a very small scale which would make it continuous but if you open any weather report, they'll often only be recorded every few minutes or hours. This is often enough to know general questions such as, "Was it raining last tuesday?", "Did it go below 0 today?", but suppose you wanted to know exactly what temperature it was at 7:23, 4 seconds in 2 weeks ago. If you're lucky, there might have been a temperature recorded at exactly that time but otherwise we might have to average from nearby times to get an _estimate_. Hopefully this estimate will be correct enough for whatever purpose you need but it's not garunteed, maybe the clouds cleared for a minute and it got significantly hotter...we couldn't know. When we sample a continuous signal, we often lose this kind of information but first let's look at how we notate the two.

<br>

There is two sets of notation, one for continuous signals which we've seen and one for a discrete signal: 

<br><br>
$$
\begin{align}
Continuous: x(t) & \quad t \in [0, T_{total}], \quad & t \in \mathbb{R} \\
Discrete: x[n] & \quad n \in [0, N_{total}], \quad & n \in \mathbb{N} 
\end{align}
$$
<br><br>

Where we can call the time duration of the signal $T_{total}$ and the total number of samples $N_{total}$.

For example, if we wanted to take a sample every $T_s = 0.02$ seconds or at a sample rate, $F_s$, of $50Hz$, we would get: 

<br> 
$$
x[n] = x(n \cdot T_s) \\
x[0] = x(0.00), \\
x[1] = x(0.02), \\
x[2] = x(0.04), \\ 
\vdots
\\
x[N_{total}] = x(T_{total}) 
\\
$$

As we saw previously time and frequency are inversly proportional, this still holds for our time intervals and our sampling rate as they both describe time and frequency:

$$
f = \frac{1}{T}
\\
F_s = \frac{1}{T_s}
$$

Lets see what sampling a signal gives us.


In [ ]:
f = 15
t = np.linspace(0, 1, 256); # Create 256 samples of sin to act as our 'continuous' signal
y = np.sin(f * 2*np.pi * t)

original_signal = Signal(t, y)

from plot import sampling_a_signal
sampling_a_signal(original_signal)

## Sampling & Limits 

<br>

#### How much do we sample x(t)?

<br>

Well it's not a very straight forward answer unfortunatly and there's a few question we should ask.

<br>

* How many samples $N_{total}$ should you take? 
* What should be the interval between samples, $T_{s}$ and should it be uniform? 
* Do you need to sample the entire signal up to  $T_{total}]$?

<br>

The answers are largely dependant on the  signal $x(t)$ itself. 

<br><br>
Some things to consider
* The more samples we take, the better view of the underlying signal, $x(t)$ that we get. We'll see in a moment that higher frequincies require us to take more and more samples. 
* If you're trying to look at a periodic signal, you'll most likely want to capture at least one full period. It would depend on what you need to know about it.
* Music files are generally sampled uniformly as it makes playback easier and no region of the signal is considered more important than any other. If however you have a signal with important regions, you'll generally want to sample highly around those areas
<br> 

<br>

### Finding a spike in the data

If there is some random spike in your data, you may need to take many more samples before you even become aware of it or what it looks like!

In [6]:
f = 15
t = np.linspace(0, 1, 256); # Create 256 samples of sin to act as our 'continuous' signal
y = np.sin(f*2*np.pi*t)
original_signal = Signal(t, y)
original_signal.add_random_spike(size=5, strength=1)
original_signal.add_random_spike(size=10, strength=2)


from plot import sampling_a_signal
sampling_a_signal(original_signal, hide_original=True)

interactive(children=(IntSlider(value=4, description='samples', max=256), Output()), _dom_classes=('widget-int…

#### Limits - What if $F_S$ keeps going to $\infty$, what happens to the interval $T_s$?

<br> 

Our interval gets so small that eventually what we can say:

<br>

$$\lim_{F_s\to\infty} \frac{1}{F_s} = T_s = 0$$

<br>

This statement tells us that as $F_s$ gets closer and closer to  $\infty$, we get to smaller and smaller intervals, $T_s$. We'll be using limits here and there so keep this notation in mind.

<br>

Practically though, in the real world when dealing with digital signals, there is a limit to how fast we can sample signals.

#### Seems more samples gives a better approximation?
 <br>
 Exactly! In fact if in the limit, as you take more and more samples, eventually our sampeld signal gives us back the continuous signal
 
 <br><br>

$$
\lim_{F_s\to\infty} \frac{1}{F_s} = T_s = 0 \implies x[n] = x(t)
$$

<br><br>

#### What about the other way? What if we take very few samples?

<br>

In the plot above you might have noticed that at very low $F_s$, our red line approximation becomes very rough before eventually it doesn't even really resemble our orignal signal at all. The __Nyquist frequency__ is the minimum rate at which you can sample a signal without introducing any errors:

$$
F_{nyquist} = 2 \cdot f_{highest}
$$

<br>

This means if the signal has a frequency component of $10Hz$, we would need to sample our signal at $F_s = 20Hz$ at the minimum or else we are _under sampling_ the signal.

#### Anything else to consider when sampling?

<br>

Well hopefully it should be quite clear that we also need to have sampled at least one period of the signal. If you are trying to detect very low frequencies below $1Hz$, you will have to take samples for at least $T$, the period of the signal. No matter how much you increase $F_s$ it will still look like the low frequency component is just a general trend in your signal rather than a periodic signal itself.

<br>

One more issue that we have to consider is _Aliasing_

## Aliasing

<br>

Aliasing refers to the fact is we take some signal at a frequency $f_0$, when sampled at some $F_s$, there are an infinite amount of other frequencies which will give the exact same samples.

<br>

This means we might accidentally detect some high-frequency signal but mistake it as a low frequency signal.

For a __sin wave__ the aliasing frequencies, $f_{alias}$, of other sins are:

<br>
$$f_{alias} = f_0 + (k \cdot F_s) \quad k \in \mathbb{N}$$ 
<br>

It seems from this that the more we sample our signal, the higher frequency has to be to pose as an alias!
You'll see in the below animation that the aliases all have the same y values at every sample we take, giving the illusion of our original signal.


In [26]:
from plot import aliases_figure
f0 = 2
fs = 5

aliases_figure(f0, fs)

## Noise

In the real world, signals are often _noisy_. That means there is some disturbance that is corrupting the signal. When the noise is not very powerful, we often don't notice but sometimes the noise corrupting the signal can be so much that we can't recognize the underlying signal anymore. If you've listened to music in a queit room, you can make out the music very clearly but if you we're to suddenly introduce a party full of people, you might have to turn up the music to be able to make it out. Another example of noise is when the rovers on Mars sends a kind radio signal back to earth but often the radiation in space can interfer with the messages sent back. When transmission can take up to 20 hours to transmit, you'll want to minimize noise as much as possible!

We can define noise as its own random signal (it might not always be so random though so be aware) that is added onto our signal.

$$
x_{noisy}(t) = x(t) + noise(t)
$$

Let's see what noise looks like on a signal!

In [67]:
f = 4
fs = 256
t = np.linspace(0, 1, fs)
y = 2 * np.sin(f * 2 * np.pi * t)
sig = Signal(t, y)

sig.add_noise(strength=2, std_dev=0.5)
sig.update_noise_line_opts({'legend': 'noisy signal'})
sig.update_line_opts({'legend': 'underlying signal'})

sig.remove_view('line') # Comment this out to see the underlying signal

sig.show(size=(1600, 300), animated=True, frames=128); 

# __Detecting Signals__

<br>

We're going to try and investigate how to detect these periodic signals we've been messing around with. If you were given the data of a discrete signal $x[n]$, how might you work out the frequency of the signal that produced it?

# Frequency Estimation, the problem
<br>

Let's create a random signal to illustrate the problem at hand!

In [106]:
fmin = 1
fmax = 10
fs = 256

f = np.random.randint(fmin, fmax+1)
t = np.linspace(0, 1, fs)
y = np.sin(f * 2*np.pi * t)
random_sig = Signal(t, y)

random_sig.show(size=(1800, 300), title='Random Signal');


What frequency is this signal at?

<br>

#### Counting the amount peaks

A manual way might be to count all the peaks between 0 - 1 seconds. This works well for small $f$ and a computer could handle those for larger $f$. Problem solved!

<br>

But what if our random signal was a step function? $x[n] = 1$ for the first half of the samples, until some point midway point $a$ and then 0 for the rest, we would get the result that $f = N_{total} / 2$ which is incorrect. Another issue might be when we start introducing more components, we will completly miss all the small frequencies!

<br>

A better method is to try and _match_ the signal with different frequencies. 

<br>

Lets make 9 signals from with $f = [1, 9]$ and compare them visually.

![step_function](images/step.png)

In [107]:
fmin = 1
fmax = 10
fs = 256

f = np.random.randint(fmin, fmax+1)
t = np.linspace(0, 1, fs)
y = np.sin(f * 2*np.pi * t)
random_sig = Signal(t, y)
random_sig.show(size=(1800, 300), title='Random Signal');

from plot import signal_grid
possible_signals = [Signal(t, np.sin(freq*2*np.pi*t), frequency=freq) for freq in range(fmin, fmax+1)]
signal_grid(possible_signals)

#### A computational way

<br>

Perhaps visually we can identify what the frequency but a computer can't. What we need is a numerical solution! Ideally we would like some measuring tool that measures how much a frequency is contained in the signal, giving us a large number when they match and ideally a 0 when they don't. 

<br>

We've already seen that adding signals just gives us another signal and that even if we added it all up, we still get just as much negative as we do positive, giving us a $0$, no good. A better option might be to multiply the two signals together, that way the negative troughs turn into positive peaks! What about if they don't match though?

## The Dot Product Operation

<br>

### Vector

A vector is commonly just a list of numbers, for example $\textbf{x} = [2, 3, -3.9, 346]$ is a vector where all its elements $x \in \mathbb{R}$.

<br>

You can see that a _vector_ looks very similar to how we might describe our discrete signals as that's exactly what our discrete signals are, a long list of numbers that describes the signal. If you like, you could also think of our continuous signals $x(t)$ as infinitely large vectors!

<br>

### Dot Product - Summation

This dot product works on two vectors of number and gives us back a scaler, a single number, that gives us a measure of how much the vectors move in the same direction and how big they are. 

For $\textbf{x}_1 = [2, 3]$ and $\textbf{x}_2 = [1, 4]$, two vectors in __2-dimensional space__:

<br><br>
$$
\begin{align}
\langle \textbf{x}_1 ,\textbf{x}_2 \rangle & = \sum_{n=0}^{N-1} \textbf{x}_1[n] \cdot \textbf{x}_2[n]  
\\
& = 2 \cdot 1 + 3 \cdot 4
\\
& = 14  
\end{align}
$$
<br><br>

Try line up the two vectors so that they're perpindicular and see the result of the dot product! You'll notice when they're at $90^{\circ}$ to each other, $\langle \textbf{x}_1 ,\textbf{x}_2 \rangle = 0$.

In [ ]:
# Create plot of adding two vectors and their projections


#### How does this help us?

<br>

Well our two discrete signals can describe two giant vectors in $N$-Dimensional space. This might be a little hard to envision with the kinds of $N$ we are dealing with here so there is a much easier interpretation for our periodic signals. It tells us how a like the frequencies of our two signals are. In fact it can tell us if they are at the same frequency or not quite easily. Lets consider two sampled $sin$ signals,$s_1$, $s_2$ with some amount of samples between $[0, 2\pi]$.

$$
\textbf{s}_1[n], \quad \textbf{s}_2[n] \quad both \; with \; length \; N
$$

As we increase our sampling rate $F_s$, the total amount of samples $N$ goes up and the closer we are to the real continuous signal as we saw before when talking about limits!
If we look at the $\lim_{F_s \to \infty}$, our discrete signals $\textbf{s}$ approach the continuous signals $sin(t)$. These means that taking our dot product between them is the same as simplying multiplying $sin(x)$ with $sin(x)$ and taking the integral over them. This is all summed up below but don't get too bogged down in it, we're simply showing that the dot product has a continuous and a discrete representation.


<br><br>
$$
\begin{align}
\lim_{N \to \infty}<\textbf{s}_1, \textbf{s}_2> \;
& = \sum_{n=0}^{N}\textbf{s}_1[n] \cdot \textbf{s}_2[n]
\\
& = \int_0^{2\pi}sin(x) \cdot sin(x) \; dx
\\
& = \int_{0}^{2\pi} sin^2(x) \; dx
\end{align}
$$
<br><br>

In [ ]:
# Create plot similar to sampling one but show sin(x) * sin(x) = sin^2(x) allow display the difference between the discrete version and the continuous one, emphasise approximation

#### ...and?

Well the intergral  $\int_{0}^{2\pi} sin^2(x) \; dx \ne 0$ as its positive everywhere, meaning they are '_pointing in a similar direction_' which means our dot product $<\textbf{s}_1, \textbf{s}_2> \ne 0$.

<br>

An interesting fact is that if you have two sin signals at __different integer frequencies__, $n$ and $m$, the result of a dot product will always be 0!

<br>

Using these facts, we can compare if two signals are at the same frequency just by taking their dot product. If we get an answer $\ne 0$ then we have a match! As long as we take enough samples to capture all the frequency information in the signal, the __Nyquist Frequency__, then this should hold true for our discrete signals too!
<br>


_When working with smaller numbers with lots of decimal points, a computer can make rounding errors so they may not be exactly 0 but they will be very close to it_

<br>

What's remarkable is that even if the signal is buried with other signals or noise, this method lets us estimate how much of that frequency content is in there. 

<br><br>
$$
\int_{0}^{2\pi}sin^2(x) \; dx = \pi
$$
<br><br>

<br><br>
$$
\begin{align}
<sin(nx), sin(nx)> \; & = \int_{0}^{2\pi} sin(nx) \cdot sin(nx) 
\\
&= \int_{0}^{2\pi} sin^2(nx) \; dx = \pi 
\\
\\
<sin(nx), sin(mx)> \; & = \int_{0}^{2\pi} sin(nx) \cdot sin(mx) = 0 
\end{align}
$$
<br><br>


#### What does the number (scalar) given by the dot product measure?

<br>

It gives us a measurement of how much each dimension lines up with eachother, each dimension just being some sample $n$ of $x[n]$. The more indexes we introduce, the higher this value will be if the signals line up. We might also __normalize__ this so we can compare them across different amounts of total samples $N$. 


<br><br>
$$
normalized \; value = \frac{1}{N}<\textbf{s}_1, \textbf{s}_2>
$$
<br><br>


## Using the dot product

Lets now use the dot product to try and see what frequency the signal is at and see what happens!

In [ ]:
# Same 3x3 grid but with normalized dot product beside them

## Composite signals
We can even see how much a certain frequency is present in a signal composed of multiple underlying frequencies! Let's try adding three signals together and see the what happens.

$$
x(t) = \sum_{n=0}^{3} x_n(t)
$$

In [ ]:
# Same but composite random signal

## Noisy Signals
Lastly, this even works for noisy signals! Lets add some noise into our signals and see how it fairs! 

In [ ]:
# Same but noisy signal

# Dealing With Phase

<br>

## Phase

<br>

This whole time we've been conveniently glossing over another component of trigonometric functions, __phase__. Phase describes the starting point of a signal in its period $T$ with respect to some point (0 is a fairly good starting point).

<br><br>
$$ x(t) =  A \cdot sin(x + \varphi)$$
<br><br>

$A$ is our amplitude as we've seen before, $f$ is the frequency and the last symbol $\varphi$ ("phi") is known as the _"phase shift"_ or _"phase offset"_. 

<br>

This helps us relate the two well known functions, $sin(x)$ and $cos(x)$ as $cos(x + 90^{\circ})$  = $sin(x)$

<br>

It's quite common to talk about phase in terms of radians but if you need to convert from degrees to radians, we can convert using the equation $ deg^{\circ} * \frac{\pi}{180} = radians$ , $360^{\circ} = 2\pi$.

<br> 

Normally we talk about phase in a $2\pi$ range, $[0, 2\pi], [-\pi, \pi], [-180^{\circ}, 180^{\circ}]$ because our signals are periodic about this range.

<br><br>
$$
sin(x + 0) = sin(x + 2\pi),  \quad sin(x - \pi) = sin(x + \pi)
$$
<br><br>



Hopefully the below graphic helps clear things up

In [ ]:
# Phase interactive plot

## Identifying the problem

<br>

Well we no longer have the nice property of $sin(x)\cdot sin(x) = sin^2(x)$ so we wouldn't expect as strong a result from our dot product.

<br>
$$
\int_{0}^{2\pi}sin(x) \cdot sin(x + \varphi) \ne \int_{0}^{2\pi}sin^2(x)
$$
<br>

Let's see how phase affects the product of our two signals.



In [ ]:
# Interactive phase multiplication


#### So our detector won't work anymore?

<br>

While it won't give as strong a response as it did when the signals were in phase, we still do get a recognisable spike in a certain frequency, although it can now be negative. You may have seen from the phase plot above that when we have similar frequencies but they are exactly $\frac{\pi}{2}$, we will end up with a dot product of 0. This means that frequencies might slip through or detector un-noticed. 

<br>

Let's introduce a random phase $\varphi \in [\frac{\pi}{2}, \frac{3\pi}{4}]$ into our random signal!

In [ ]:
# Random signal with phase now

Let's look under the hood and see how our detector is fairing step by step. 

In [ ]:
# Multiply signals and take dot product

<br>

So as we saw, the dot product, $<sin(x), \, sin(x + \pi/2)> \, = 0$, which means even though they are at the same frequency they aren't measured as similar. From this we can say that the dot product not only measures if the signals are at the same frequency but it can also tells us how _in phase_ two signals are. For two signals to be _in phase_ they have a phase offset of 0 from one another.

# Forming a solution

<br> 

If you havn't already, go back and try with `phase_offset = np.pi`, what you will notice is that we get a dot product that is just as large but negative and it should be hopefully be clear why that is. Let's graph the the __dot product vs $\varphi$__ and see how they relate.

<br> 

In [ ]:
# Plot dot product vs phase

As we can see from our plot, our dot product moves smoothly as the phase changes. This plot also highlights that our dot product is 0 at a phase offset of $\pm  \frac{\pi}{2}$ as we saw before.

<br>

So lets try detecting with a signal $sin(x + \frac{\pi}{2})$ and overlay that and see what we get.

<br>

... before we do though, we should try consider what we're looking for. We know that if they're in phase, we get a peak in our dot product graph. Ideally we would like some way to have our entire graph be at a the peak so for any value of $\varphi$, we get get a strong reponse when using the right $f$ in our detector.

<br>

Also you might remember we showed $cos(x) = sin(x + \frac{\pi}{2})$ so we will use that instead.

In [ ]:
# Dot product with sin and cos

Okay this looks a little more promising, where our dot product with $sin(x) = 0$ , $cos(x)$ is at one of the positive or negative peaks.

<br>

#### Sure but how can we combine their results to get the true dot product?

<br>

Lets try adding them first?


In [ ]:
# Adding the cos and sin signals together and plotting their result

Multiplying won't work either because the $0$'s at $\pm\pi \,/ \, 2$ will cancel out the value for the other detecting signal. The addition seems kind of promising but the negative values cancel out with the positive ones. What if we tried adding their absolute values so they're all positive, after all, we only care about the overall amplitude, how strongly we detect a signal.  

In [ ]:
# Add their absolute values

Whoops, well we got what we wanted and we definitely have a strong response no matter what $\varphi$ but we get a lumpy pattern that isn't really correct. While this would work for our detector, to be accurate we should be expecting a straight-ish line.

<br>

Lets try the usual method of distance we use in trigonometry $distance = \sqrt{x^2 + y^2}$  or in our case, $\sqrt{ (cos \; dot \; product)^2 + (sin \; dot \; product)^2}$, the squaring should get rid of any negative values and the square root will bring our magnitude back down after squaring the values. 

In [ ]:
# Plot using the distance formula isntead this time

It took some investigation but we eventually got there! We'll see later why the last method works so well but for now we'll accept that we got there through trial and error.

In [ ]:
# Detector plot but with distance measurements, 3x3 grid with both sin and cos on them with the dp above the plot

## A quick recap
<br>

To summarise how we got here:

<br>

* We first tried a range of frequencies and took the dot product:
$$<\textbf{x}_1, \textbf{x}_2> \;= \sum_{n=0}^{N-1}\textbf{x}_1[n] \cdot \textbf{x}_2[n]$$
Where $\textbf{x}_1$ is our random signal we sampled and $\textbf{x}_2$ was some $sin(2\pi f)$ signal we sampled

<br>

* We tried a range of frequencies for $f$ and the one that gave the highest dot product would be the frequency we said our random signal was closest too.

<br>

* We also then made our random signal $\textbf{x}_1$ more complex by introducing more frequency components and saw that our detector was able to pick out the individual components too.

<br>

* After adding a random phase into our signal component, we saw that our detector no longer worked as the negative and positive parts of $\textbf{x} = \textbf{x}_1 \cdot \textbf{x}_2$ would cancel out. When we added them all up we would get a value that was anywhere between our maximum and minimum as show in the __dot product vs 𝜑__ plot.

<br>

* We saw that $<sin(x), \, sin(x + \pi/2)> \, = 0$, which led us to also try detect with $sin(x + \pi/2)$ or more simply $cos(x)$. When we combined the two dot products made by these waves with our signal, we could get an accurate answer even if the signal was not _in phase_ with our detector.

<br>

#### What now?

<br>

We can arrive at a much neater solution but first we need to talk about __Complex numbers__ and __Orthogonality__ a little bit.


# A Complex Introduction

<br>

## Orthogonality

<br>

When two lines or curves, $x, y$,  have a  dot product $\langle x,y \rangle = 0$
This means they are orthogonal. For two vectors in our x, y plane that we saw earlier this meant they were perpindicular to each other.

<br>

For our waves we saw that when the frequencies were integers and didn't match or when they were $90^{\circ}$ out of phase, we got a dot product of $0$. 

<br>

#### So....?

<br>

Well orthogonal signals slipped through so we definitely have to try all frequencies but how would we capture the phase? Using $sin(nx)$ and $cos(nx)$ will allow us to capture the information in that frequency, detecting our random signal that could be of any phase. 

<br>

A comparison you could use is that in our x,y plane, using two perpindicular vectors allows us to describe any point in that plane by combining those two vectors. 

<br>

#### Couldn't you just use any two vectors as long as they aren't pointing in the same direction?

<br>

You could but using orthogonal vectors opens up a whole range of mathematical ideas that are based upon orthogonality as well as simplfy calculations. As we'll see later, this is where we can get our maginitude formula that we used earlier $\sqrt{x^2 + y^2}$

<br><br>
#### Orthogonal vectors

$$
\langle \textbf{x}_1, \textbf{x}_2 \rangle = \sum_{n=0}^{N-1} \textbf{x}_1[n] \cdot \textbf{x}_2[n] \, = 0  
$$

<br><br>

#### Orthogonal periodic functions

$$
\begin{align}
\langle sin(nx), sin(mx) \rangle = \int_{0}^{2\pi}sin(nx) \cdot sin(mx) \, dx &= 0,\; n\ne m, \quad n,m \in \mathbb{N}
\\ 
\langle sin(nx), cos(nx) \rangle =\int_{0}^{2\pi}sin(nx) \cdot cos(nx) \, dx &= 0  
\end{align}
$$

<br>


# Complex Numbers

<br>

## A basic introduction

<br>

Using complex numbers we can combine both the $sin$ and $cos$ signals in our detector into just one complex one and use that to detect signals.

<br>

#### Complex numbers, the imaginary ones with $i$?

<br>

If you're comfortable with complex numbers feel free to skip ahead but we want to try build an intuition to the point that $\int f(x) \cdot e^{i\theta}$ makes sense and that $e^{i\theta}$ is the building block of signals or a basis function of signal space.

<br>

## A quick recap

<br>

We represent an imaginary number as $i$ = $\sqrt{-1}$. If we can imagine expanding the real number line into two dimenions, horizontally and vertically then what we get is the complex plane, where real numbers $(\mathbb{R})$ lie on the horizontal axis and imaginary numbers $(i)$ on the vertical one. All points in this plane are complex numbers and consist of a real number and an imaginary one.

$z$ is quite a common symbol to denote a complex number. 

$$
z = a + bi
$$

Here $a$ is our real part and $bi$ is our imaginary part. This gives us our complex number in what is known as __rectangular__ form. Its distance from the origin $(0,0)$, $r = \sqrt{a^2 + b^2}$. We can also calculate the angle it makes from the real axis using any one of the trig identities such as $tan(\theta) = \frac{b}{a}$. We won't be doing much converting so if you are rusty don't worry, this is more to get a feel for them.

<br>

Another representation of a complex number which can look like a vector in the complex plane is to describe it using the __angle__ $\theta$ it makes with the real axis, $\theta$ as well as the __magnitude__ or size of the vector, $r$.


In [ ]:
%%html
<div style="display: flex; justify-content: center">
<iframe width="1022" height="575" src="https://www.youtube.com/embed/SP-YJe7Vldo" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
</div>

Note: All Khan Academy content is available for free at (www.khanacademy.org). Feel free to go further and learn more if this is all new to you!

In [ ]:
# Plot to show the rectangular vs polar

#### How do we write one down in polar form?

<br>

Well some trigonometry will tell us that $a = r cos(\theta)$ and $b = r sin(\theta)$ 

<br>

#### I noticed a cos and sin in there, is that related to our detector?

<br>

Absolutely! In fact when we computed the dot product of our signal in our detector, we were also computing a complex number $z = a + bi$, the $a$ and $b$, the real and imaginary parts of a complex number. The strength of our $cos$ wave signal output is actually just the length of $a$ and likewise for $sin$ and $b$! Lets show what we mean:


In [ ]:
#TODO:
# Show in rectangular, highlight rect form as being cos and sin parts
# Show in polar form, highlight polar form as being the overall magnitude and phase

<br>

Remember we figured out how to combine our cos and sin outputs to get the overall dot product no matter the phase? In effect we were computing $r$ the magnitude of our response with our combination of $\sqrt{cos \, dot \, product^2 + sin \, dot \, product^2} = \sqrt{a^2 + b^2}$. 

<br>

Let's work in that improvement now and instead of plotting the rows for our $cos$ and $sin$ signals as we did before, changing their background colours lets instead combine them together into the one plot! We'll only use 4 frequencies to save space and get a clearer picture of whats going on.

<br>

In [ ]:
# 2 x 2 grid of complex detector

<br>

Turns out we can actually compute the phase quite easily too, the phase of our signal we were trying to detect is encoded as the angle of our complex number $\theta$. We can get this angle with some more trig functions $tan(\theta) = \frac{b}{a}$.

In [ ]:
# TODO: Come up with a nice display of output, add arc for angle, changeb title 

#### $e^{i\theta}$

Starting with work from Bernoulli and later expanded upon by Leonard Euler, we arrive at a hugely useful equation that $e^{i\theta} = cos(\theta) + i \cdot sin(\theta)$.

<br>

This lets us express cos and sin terms even more compactly as just $e^{i\theta}$ giving us our final polar form as a __complex exponential__ which you can see used widely through signal processing!



We can get rid of the extra letter and just use $\lvert z \rvert$ to signify the absolute value or the magnitude of our complex number.

<br>

We recommend checking out this video below by 3Blue1Brown if you want to get a better sense of how these work but by no means required!

In [ ]:
%%html
<div style="display: flex; justify-content:center">
<iframe width="1280" height="720" src="https://www.youtube.com/embed/mvmuCPvRoWQ" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
</div>

## Summary 

<br>

* We used two orthogonal functions $sin(nx)$ and $cos(nx)$ to try and detect a signal at frequency $n$. 

* We also tried and failed with a load of $sin(mx)$ and $cos(mx)$ because they were orthogonal to our signal at frequency $n$.

* The two dot product values that were related could compactly be stored as a single number $z = a + bi$

* Complex numbers have another way to be represnted and that's in polar form which we saw encapsulated or $cos$ and $sin$. 

* This $cos$ and $sin$ term could be neatly packed into a __complex exponential__ $\lvert z \rvert\ e^{i\theta}$

* Finally we saw geometrically how $cos$ and $sin$ relate to our complex exponential
<br>

#### A brief aside on notation

<br>

In engineering $i$ normally stands for eletrical current so you might sometimes see $\sqrt{-1} = j$ instead. Another common thing to see is using $e^{-i\theta}$. This effects the direction of our frequency as you can check out in the rotating complex plots if you need. Practically this doesn't make a difference as our frequencies remain the same. 

#### Using $e^{i\theta}$

Well our old detector was taking a dot product between the sampled signal and our $cos$ and $sin$, our two __basis functions__. We can now instead just use the one basis function $e^{i\theta}$ sampled at the same interval and get all the information we need from that! 

<br><br>
$$
\langle f[n] ,\, e^{i\,2\pi f nT_s} \rangle = \sum_{n= 0}^{N-1} f[n] \, e^{i\,(2\pi f) \, (nT_s)}
$$
<br><br>

It can be helpful to think of $f[n]$ as being our magnitude $r$ and the $2\pi f$ term as being our $\theta$. I like to think of $cos(\theta)$, $sin(\theta)$ and $e^{i\theta}$ as being three different windows for looking in at our underlying frequency $\theta$. 

This let's us read the above equation as "sum up all the complex numbers $r e^{i\theta}$ to get our dot product" 

<br>

_It's worth reiterating again that $2\pi f$ let's us convert to Hz and stands for our frequency $\theta$ and that $nT_s$ lets us sample a continuous signal at the same points of time as our sampled signal f[n]_. 
<br>

# The Fourier Series

<br>

The Fourier Series, the discovery of which is credited to Joesph Fourier, a French Mathematician of late 1700's / early 1800's is a mathematical idea that we can break up any periodic function into a sum of inifite sines and cosines. 

$$
f(x) = a_0 + \sum_{n = 1}^{\infty} a_n cos(nx) + \sum_{n = 1}^{\infty} b_n sin(n x)
$$

Where $f(x)$ is some periodic function that we can express in terms of a set of coefficients, $a_0, a_n, b_n$.

<br>

What this equation tells us is that we can express any function that repeats itself as a sum of $cos$ and $sin$ terms ( all be it possibly infinitly many of them ). __TODO: Create a better image__

<br>

![fourierapprox](images/fourierseries.svg.png)

<br>

While possibly daunting to try consider at first, we can go back to the fact $cos(nx)$ and $sin(nx)$ are orthogonal, allowing us to investigate a certain frequency $n$, no matter the phase. We also saw that $sin(nx)$ is orthogonal to $sin(mx)$ and so each component $n$ investigates a different freqeuncy of $f(x)$.
If alarm bells are rining to use $e^{in}$ then you're right! Well first see what $a_0$ is about.

#### So what about the $a_0$ term?

This term is the mean amplitude of our function, or the mean of a signal. 

<br>

There is a general equation for the mean of a function

$$
function \; average = \frac{1}{b - a}\int_{a}^{b} f(x) \; dx 
$$

#### Mean value of a $2\pi$ periodic function

<br><br>
$$
a_0 = \frac{1}{2\pi}\int_{0}^{2\pi}f(x) \, dx 
\quad
or
\quad
a_0 = \frac{1}{2\pi}\int_{-\pi}^{\pi}f(x) \, dx 
$$

<br><br>

![means](images/means.png)
<br>

We can see that $a_0$ is equal to adding up every point in f(x) and finding their average over the length of the period $2\pi$. You'll notice that the limits are different but the size of the interval is still the same. Take a moment to make sure that makes sense.


It is for now but what this formulation will allow us to do is use complex numbers to encapsulate all of this into a single equation:

<br>
$$
\begin{align}
f(x) & = a_0 + \sum_{n = 1}^{\infty} a_n cos(nx) + \sum_{n = 1}^{\infty} b_n sin(nx)
\\
f(x) & = \sum_{n = -\infty}^{\infty}c_ne^{i nx}
\\
\\
where \quad c_n & = \frac{1}{2\pi}\int_{-\pi}^{\pi}f(x)e^{-inx} dx
\end{align}
$$
<br>

This is called the __Complex Fourier Series__ and while more complex to derive, it makes calculations much simpler and allows us to not only analyse real signals as we have been doing but also complex ones.

<br>
We won't do the derivation here but for those a little more interested, here's a complete derivation.


In [ ]:
%%html
<div style="display: flex; justify-content: center">
<iframe width="1280" height="720" src="https://www.youtube.com/embed/Ft5iyapkSqM" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
</div>